In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.1 MB/s 
     |████████████████████████████████| 212 kB 71.8 MB/s 
     |████████████████████████████████| 115 kB 72.5 MB/s 
     |████████████████████████████████| 163 kB 92.4 MB/s 
     |████████████████████████████████| 127 kB 84.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.2 MB/s 
     |████████████████████████████████| 182 kB 63.2 MB/s 
     |████████████████████████████████| 166 kB 60.9 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 166 kB 62.4 MB/s 
     |████████████████████████████████| 162 kB 99.3 MB/s 
     |████████████████████████████████| 162 kB 70.2 MB/s 
     |████████████████████████████████| 158 kB 67.2 MB/s 
     |████████████████████████████████| 157 kB 70.9 MB/s 
     |████████████████████████████████| 157 kB 63.4 MB/s 
     |████████████████████████████████| 157 kB 69.2 MB/s 
     |████████████████████████████████| 157 kB 91.3 MB/s 
     |████████████████████████████████| 157 kB 80.8 MB/s 
     |████████████████████████████████| 157 kB 86.1 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████

In [ ]:
!pip install transformers==4.21.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 34.6 MB/s 


In [ ]:
import wandb
import os
from datasets import DatasetDict, load_dataset, load_metric

In [ ]:
USER = "siddhantm"
WANDB_PROJECT = "capstone_jpmc"
wandb.init(entity=USER, project=WANDB_PROJECT)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# PROJECT_ROOT = "/content/drive/MyDrive/JPMC Capstone/Speech_Emotion"
PROJECT_ROOT = "/content/drive/MyDrive/JPMC Capstone/Audio-Classification-HF-main"

In [ ]:
!pwd

/content


In [ ]:
os.chdir(PROJECT_ROOT)

In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1_cBqTwetZCYD3uVjVHeosGtR7NkaEAwE/JPMC Capstone/Audio-Classification-HF-main


In [ ]:
import os
import logging
import librosa
# import src.utils

import numpy as np

from datasets import DatasetDict, load_dataset, load_metric
from transformers import (
    HubertForSequenceClassification,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    Wav2Vec2FeatureExtractor,
)
from src.utils import collator

logging.basicConfig(
    format="%(asctime)s | %(levelname)s: %(message)s", level=logging.INFO
)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor
model = "facebook/hubert-base-ls960"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model)
print(feature_extractor)

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}



In [ ]:
NUM_LABELS = 6
model_id = "facebook/hubert-base-ls960"

# config = PretrainedConfig.from_pretrained(model_id, num_labels=NUM_LABELS)
# hubert_model = HubertForSequenceClassification.from_pretrained(
#     model_id,
#     config=config  # because we need to update num_labels as per our dataset
#     # ignore_mismatched_sizes=True,  # to avoid classifier size mismatch from from_pretrained.
# )

hubert_model = HubertForSequenceClassification.from_pretrained(model_id, num_labels=NUM_LABELS)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['projector.weight', 'classifier.weight', 'projector.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# freeze all layers to begin with
for param in hubert_model.parameters():
    param.requires_grad = False
    
# freeze two encoder layers    
layers_freeze_num = 2
n_layers = (
    4 + layers_freeze_num * 16
)  # 4 refers to projector and classifier's weights and biases.
for name, param in list(hubert_model.named_parameters())[-n_layers:]:
    param.requires_grad = True

In [ ]:
dataset_config = {
  "LOADING_SCRIPT_FILES": os.path.join(PROJECT_ROOT, "src/data/crema.py"),
  "CONFIG_NAME": "clean",
  "DATA_DIR": os.path.join(PROJECT_ROOT, "data/archive.zip"),
  "CACHE_DIR": os.path.join(PROJECT_ROOT, "cache_crema"),
}
ds = load_dataset(
  dataset_config["LOADING_SCRIPT_FILES"],
  dataset_config["CONFIG_NAME"],
  data_dir=dataset_config["DATA_DIR"],
  cache_dir=dataset_config["CACHE_DIR"]
)
print(ds)

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'label'],
        num_rows: 7442
    })
})


In [ ]:
# CONVERING RAW AUDIO TO ARRAYS
ds = ds.map(
        lambda x: {
            "array": librosa.load(x["file"], sr=16000, mono=False)[0]
        },
        num_proc=2,
    )

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'label', 'array'],
        num_rows: 7442
    })
})


In [ ]:
# PROCESS THE DATASET TO THE FORMAT EXPECTED BY THE MODEL FOR TRAINING

INPUT_FIELD = "input_values"
LABEL_FIELD = "labels"

def prepare_dataset(batch, feature_extractor):
    audio_arr = batch["array"]
    input = feature_extractor(
        audio_arr, sampling_rate=16000, padding=True, return_tensors="pt"
    )

    batch[INPUT_FIELD] = input.input_values[0]
    batch[LABEL_FIELD] = batch[
        "label"
    ]  # colname MUST be labels as Trainer will look for it by default

    return batch

In [ ]:
# APPLY THE DATA PREP USING FEATURE EXTRACTOR TO ALL EXAMPLES
ds = ds.map(
    prepare_dataset,
    fn_kwargs={"feature_extractor": feature_extractor},
    # num_proc=2,
)
logging.info("Finished extracting features from audio arrays.")

In [ ]:
# LABEL TO ID
ds = ds.class_encode_column("labels")

Casting to class labels:   0%|          | 0/8 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'label', 'array', 'input_values', 'labels'],
        num_rows: 7442
    })
})


In [ ]:
# INTRODUCE TRAIN TEST VAL SPLITS

# 90% train, 10% test + validation
train_testvalid = ds["train"].train_test_split(shuffle=True, test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'val': test_valid['train']})

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'label', 'array', 'input_values', 'labels'],
        num_rows: 6697
    })
    test: Dataset({
        features: ['file', 'label', 'array', 'input_values', 'labels'],
        num_rows: 373
    })
    val: Dataset({
        features: ['file', 'label', 'array', 'input_values', 'labels'],
        num_rows: 372
    })
})


In [ ]:
trainer_config = {
  "OUTPUT_DIR": "results",
  "TRAIN_EPOCHS": 5,
  "TRAIN_BATCH_SIZE": 8,
  "EVAL_BATCH_SIZE": 8,
  "GRADIENT_ACCUMULATION_STEPS": 4,
  "WARMUP_STEPS": 500,
  "DECAY": 0.01,
  "LOGGING_STEPS": 10,
  "MODEL_DIR": "models/test-hubert-model",
  "SAVE_STEPS": 100
}

# Fine-Tuning with Trainer
training_args = TrainingArguments(
    output_dir=trainer_config["OUTPUT_DIR"],  # output directory
    gradient_accumulation_steps=trainer_config[
        "GRADIENT_ACCUMULATION_STEPS"
    ],  # accumulate the gradients before running optimization step
    num_train_epochs=trainer_config[
        "TRAIN_EPOCHS"
    ],  # total number of training epochs
    per_device_train_batch_size=trainer_config[
        "TRAIN_BATCH_SIZE"
    ],  # batch size per device during training
    per_device_eval_batch_size=trainer_config[
        "EVAL_BATCH_SIZE"
    ],  # batch size for evaluation
    warmup_steps=trainer_config[
        "WARMUP_STEPS"
    ],  # number of warmup steps for learning rate scheduler
    save_steps=trainer_config["SAVE_STEPS"], # save checkpoint every 100 steps
    weight_decay=trainer_config["DECAY"],  # strength of weight decay
    logging_steps=trainer_config["LOGGING_STEPS"],
    evaluation_strategy="epoch",  # report metric at end of each epoch
    report_to="wandb",  # enable logging to W&B
)

PyTorch: setting up devices


In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

import torch
from transformers import Wav2Vec2Processor

INPUT_FIELD = "input_values"
LABEL_FIELD = "labels"


@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(
        self, examples: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:

        input_features = [
            {INPUT_FIELD: example[INPUT_FIELD]} for example in examples
        ]  # example is basically row0, row1, etc...
        labels = [example[LABEL_FIELD] for example in examples]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        batch[LABEL_FIELD] = torch.tensor(labels)

        return batch

In [ ]:
# DEFINE DATA COLLATOR - TO PAD TRAINING BATCHES DYNAMICALLY
data_collator = DataCollatorCTCWithPadding(
            processor=feature_extractor,
            padding=True
)

In [ ]:
from datasets import load_metric

def compute_metrics(eval_pred):
    # DEFINE EVALUATION METRIC
    compute_accuracy_metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return compute_accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
# START TRAINING
trainer = Trainer(
    model=hubert_model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=ds["train"],  # training dataset
    eval_dataset=ds["val"],  # evaluation dataset
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

# TO RESUME TRAINING FROM CHECKPOINT
# trainer.train("results/checkpoint-2000")

The following columns in the training set don't have a corresponding argument in `HubertForSequenceClassification.forward` and have been ignored: file, array. If file, array are not expected by `HubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 6697
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 1045
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
0,1.598800,1.541855,0.384409
1,1.403300,1.219802,0.524194
2,1.264400,0.978769,0.626344
3,1.054600,0.891234,0.669355
4,0.981800,0.858751,0.642473


Saving model checkpoint to results/checkpoint-100
Configuration saved in results/checkpoint-100/config.json
Model weights saved in results/checkpoint-100/pytorch_model.bin
Saving model checkpoint to results/checkpoint-200
Configuration saved in results/checkpoint-200/config.json
Model weights saved in results/checkpoint-200/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `HubertForSequenceClassification.forward` and have been ignored: file, array. If file, array are not expected by `HubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 372
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """


Saving model checkpoint to results/checkpoint-300
Configuration saved in results/checkpoint-300/config.json
Model weights saved in results/checkpoint-300/pytorch_model.bin
Saving model checkpoint to results/checkpoint-400
Configuration saved in results/checkpoint-400/config.json
Model weights saved in results/checkpoint-400/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `HubertForSequenceClassification.forward` and have been ignored: file, array. If file, array are not expected by `HubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 372
  Batch size = 8
Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to results/checkpoint-600
Configuration saved in results/checkpoint-600/config.json
Model weights saved in results/

TrainOutput(global_step=1045, training_loss=1.294261299936395, metrics={'train_runtime': 1875.9783, 'train_samples_per_second': 17.849, 'train_steps_per_second': 0.557, 'total_flos': 1.0171783530946053e+18, 'train_loss': 1.294261299936395, 'epoch': 5.0})

In [ ]:
  # TEST RESULTS
test_results = trainer.predict(ds["test"])
logging.info("Test Set Result: {}".format(test_results.metrics))
wandb.log({"test_accuracy": test_results.metrics["test_accuracy"]})

trainer.save_model(os.path.join(PROJECT_ROOT, trainer_config["MODEL_DIR"]))

The following columns in the test set don't have a corresponding argument in `HubertForSequenceClassification.forward` and have been ignored: file, array. If file, array are not expected by `HubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 373
  Batch size = 8


Saving model checkpoint to /content/drive/MyDrive/JPMC Capstone/Audio-Classification-HF-main/models/test-hubert-model
Configuration saved in /content/drive/MyDrive/JPMC Capstone/Audio-Classification-HF-main/models/test-hubert-model/config.json
Model weights saved in /content/drive/MyDrive/JPMC Capstone/Audio-Classification-HF-main/models/test-hubert-model/pytorch_model.bin


In [ ]:
test_results

PredictionOutput(predictions=array([[ 0.73253286,  0.07682817, -1.9434329 ,  0.0857442 ,  2.2364445 ,
        -0.92333126],
       [-2.3726149 , -0.18005304,  2.9774282 , -0.03223043, -1.6441029 ,
         1.0501623 ],
       [-2.7351286 , -1.5949947 ,  3.2750595 ,  0.85149217, -1.1106035 ,
         0.91387635],
       ...,
       [-3.0244644 , -0.7129141 ,  0.824072  , -1.671285  ,  1.998154  ,
         2.304458  ],
       [-2.5450752 ,  0.69884247,  1.8545264 , -2.2018175 , -1.709566  ,
         3.324555  ],
       [-1.6346755 ,  0.5744713 , -1.71297   , -0.75079226,  3.0028322 ,
         0.789509  ]], dtype=float32), label_ids=array([4, 1, 5, 5, 4, 1, 4, 2, 3, 1, 3, 0, 5, 4, 2, 1, 3, 1, 2, 3, 1, 3,
       2, 5, 3, 5, 3, 3, 3, 4, 0, 4, 2, 0, 5, 0, 1, 1, 2, 2, 5, 1, 1, 2,
       0, 1, 3, 4, 4, 5, 4, 5, 5, 5, 0, 1, 0, 0, 1, 5, 0, 0, 1, 3, 0, 5,
       4, 5, 5, 1, 1, 2, 2, 0, 0, 2, 3, 3, 0, 1, 3, 0, 4, 2, 5, 1, 1, 5,
       4, 2, 3, 5, 4, 3, 2, 4, 5, 0, 1, 3, 2, 5, 0, 0, 2, 5, 0, 2, 0, 

In [ ]:
# logging trained models to wandb
wandb.save(
    os.path.join(PROJECT_ROOT, trainer_config["MODEL_DIR"], "*"),
    base_path=os.path.dirname(trainer_config["MODEL_DIR"]),
    policy="end",
)

ValueError: ignored